# Social Media Monitoring

## Politician List


The following table contains data on the politicians and their associated social acounts

In [ ]:
import qgrid
from smm_wrapper import SMMPoliticians
from wikiwho_wrapper import WikiWho

qgrid.set_grid_option('maxVisibleRows', 5)

# Create an instance to the smm wrapper
smm = SMMPoliticians()

# Request the politicians from the API
df = smm.dv.get_politicians()

# Show the result
qgrid.show_grid(df)

## Instructions

Please choose a politician from the table above and input the `politician_id` and `aggregation` variables in the cell below:

In [ ]:
import pandas as pd

politician_id = 240
aggregation = 'month' #aggregation can be by day ('day'), week ('week') or month('month')

# using api to extract data
wiki_chobs = smm.dv.wikipedia(_id=politician_id, from_date='2017-09-01', to_date='2018-12-31', aggregate_by=aggregation)
wiki_ids = df.loc[df.index == politician_id, 'wp_ids'].iloc[0]

#using wikiwho to extract conflicts and revisions
ww = WikiWho(lng='de')
wiki_data = pd.concat(ww.dv.edit_persistence(page_id=wiki_id) for wiki_id in wiki_ids)
wiki_data['undos'] = wiki_data['dels'] + wiki_data['reins']
wiki_data = wiki_data[['year_month','conflicts','revisions', 'undos']]

#using wikipedia and wikimedia to extract views
import wikipedia
import requests
wikipedia.set_lang('de')
session = requests.Session()
wiki_title = wikipedia.page(pageid=df.loc[df.index == politician_id, 'wp_ids'].iloc[0]).title
url = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/all-agents/{}/{}ly/20171101/20181101'.format(wiki_title, aggregation)
views = pd.DataFrame(session.get(url=url).json()['items'])
views = views[['timestamp', 'views']]

In [ ]:
#aggregating the data from wikiwho and grouping it together with chobs
wiki_data = wiki_data.groupby('year_month').agg(
    {
        'conflicts': 'sum',
        'revisions': 'sum',
        'undos': 'sum'
    }).reset_index()
wiki_data = pd.merge(wiki_data, views, left_on=pd.to_datetime(wiki_data['year_month']), right_on=pd.to_datetime(views['timestamp'], format='%Y%m%d%H')).reset_index()
wiki_total = pd.merge(wiki_chobs, wiki_data.drop(columns=['key_0','timestamp']), left_on=wiki_chobs['date'], right_on=wiki_data['key_0']).set_index('_id')

## Plotting

In [ ]:
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [go.Scatter(x=wiki_total['date'], y=wiki_total['chobs'], name='Change objects'),
            go.Scatter(x=wiki_total['date'], y=wiki_total['revisions'], name='Revisions'),
            go.Scatter(x=wiki_total['date'], y=wiki_total['views'], name='Views', yaxis='y2')], 
    "layout": go.Layout(title='Wikipedia Activity', xaxis={'title':''},  yaxis=dict(
        title='Revisions and Change Objects'
    ),
    yaxis2=dict(
        title='Views',
        
        overlaying='y',
        side='right'
    ), showlegend=True)
})

plotly.offline.iplot({
    "data": [go.Scatter(x=wiki_total['date'], y=wiki_total['undos'], name='Undos'),
            go.Scatter(x=wiki_total['date'], y=wiki_total['conflicts'], name='Conflicts')], 
    "layout": go.Layout(title='Wikipedia Conflicts', xaxis={'title':''}, yaxis={'title':'undos and conflicts'}, showlegend=True)
})